In [89]:
import os
import warnings
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo  # on windows need tzdata package

import numpy as np
import plot_utils as pu
from astropy import units as u
from astropy.coordinates import ICRS, AltAz, EarthLocation, SkyCoord
from astropy.table import QTable, unique
from astropy.time import Time
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.wcs import WCS
from astroquery.exceptions import NoResultsWarning
from astroquery.simbad import Simbad
from matplotlib import patheffects
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, LogNorm, ListedColormap, to_rgb
from timezonefinder import TimezoneFinder
from wand.image import Image

foregroundcolour = "#FFF"
blues = ['#000', "#171726", 'dodgerblue', "#00BFFF", 'lightskyblue']
magnitudes = [7, 5, 3, 0, -1]

brightness_by_time = {0:0, 3:1, 5:2, 7:3, 12:4, 15:3, 18:2, 21:1, 24:0}

all_times = np.linspace(0, 24*60, 60*24*60)
magnitude_times = [key*60 for key in brightness_by_time.keys()]
magnitude_values = [magnitudes[value] for value in brightness_by_time.values()]

magnitude_mapping = np.interp(all_times, magnitude_times, magnitude_values)

blue_mapping =[(key/24, blues[value]) for key,value in brightness_by_time.items()]
backgroundcolours = LinearSegmentedColormap.from_list("sky", blue_mapping)

tf = TimezoneFinder()
Simbad.reset_votable_fields()
generic_maximum_magnitude = max(magnitudes)

spectral_colours = {
    "O": "lightskyblue",
    "B": "lightcyan",
    "A": "white",
    "F": "lemonchiffon",
    "G": "yellow",
    "K": "orange",
    "M": "#f9706b",
    "": "white",
}
spectral_colours = {key: to_rgb(value) for key, value in spectral_colours.items()}


## User Inputs

In [90]:
location = "Toronto"
observation_point = (
    45 * u.deg,
    160 * u.deg,
)  # altitude (deg from horizon), azimuth (eastwards from north)


view_radius = 15 * u.deg

observation_start = {
    "year": 2025,
    "month": 2,
    "day": 18,
    "hour": 19,
    "minute": 0,
    "second": 0,
}
observation_length = 2 * u.hour
observation_frequency = 5 * u.minute

image_pixels = 250
fps = 2
image_directory = os.getcwd()
gif_fname = f"{image_directory}/SkySim.gif"
delete_frames = True

In [91]:
# query to a lat/long
earth_location = EarthLocation.of_address(location)
lat, lon = [l.to(u.deg).value for l in [earth_location.lat, earth_location.lon]]
timezone = ZoneInfo(tf.timezone_at(lat=lat, lng=lon))

frames = max(np.ceil((observation_length / observation_frequency).decompose()).astype(int), 1)

dt_native_start = datetime(**observation_start, tzinfo=timezone)

if frames > 1:
    dt_astropy_start = Time(dt_native_start).to_value("mjd")
    dt_astropy_end = dt_astropy_start + observation_length.to(u.day).value
    dt_astropy = Time(
        np.arange(dt_astropy_start, dt_astropy_end, observation_frequency.to(u.day).value),
        format="mjd",
    )
else:
    dt_astropy = Time(dt_native_start)

## Convert Observing Point to RA/Dec

In [92]:
# generate a coordinate frame for the observation
earth_frame = AltAz(
    obstime=dt_astropy,
    az=observation_point[1],
    alt=observation_point[0],
    location=earth_location,
)

# perform the conversion
ra_dec = SkyCoord(earth_frame.transform_to(ICRS()))

## Query SIMBAD to get a catalogue of objects with relevant data

In [93]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields("otype", "V", "ids", "sp_type")
criteria = f"otype != 'err' AND V < {generic_maximum_magnitude}"
query_result = QTable(
    Simbad.query_region(ra_dec, radius=view_radius, criteria=criteria)
)

In [94]:
# clean up the result
columns_to_remove = [
    "coo_err_min",
    "coo_err_angle",
    "coo_wavelength",
    "coo_bibcode",
    "coo_err_maj",
]
for colname in columns_to_remove:
    query_result.remove_column(colname)

# rename columns
query_result.rename_column("main_id", "id")
query_result.rename_column("otype", "object_type")
query_result.rename_column("V", "magnitude")
query_result.rename_column("sp_type", "spectral_type")

In [95]:
# round columns
query_result["ra"] = query_result["ra"].round(5)
query_result["dec"] = query_result["dec"].round(5)
query_result["magnitude"] = query_result["magnitude"].round(3)
spectral_types = []
for i in query_result["spectral_type"].data:
    if (len(i)>0) and (i[0] in spectral_colours.keys()):
        spectral_types.append(i[0])
    else:
        spectral_types.append("")
query_result["spectral_type"] = spectral_types

# create human-readable name column
query_result["ids_list"] = [i.split("|") for i in query_result["ids"]]
names_column = []
for id, namelist in zip(query_result["id"].data, query_result["ids_list"].data):
    item_names = [n[5:] for n in namelist if "NAME" in n]
    if len(item_names) == 0:
        names_column.append(id)
    elif len(item_names) == 1:
        names_column.append(item_names[0])
    else:
        names_column.append("/".join(item_names))
query_result["name"] = names_column
query_result.remove_columns(["ids", "ids_list"])

## Filter data

In [96]:
# remove child elements
parents = query_result["id"]  # check all items, regardless of type
parents_string = tuple(parents.data)
parent_query_adql = f"""
    SELECT main_id AS "child_id",
    parent_table.id AS "parent_id"
    FROM (SELECT oidref, id FROM ident WHERE id IN {parents_string}) AS parent_table,
    basic JOIN h_link ON basic.oid = h_link.child
    WHERE h_link.parent = parent_table.oidref;
"""
with warnings.catch_warnings(action="ignore", category=NoResultsWarning):
    hierarchies = Simbad.query_tap(parent_query_adql)
children = unique(hierarchies)["child_id"].data
query_result.add_index("id")
for child_id in children:
    if child_id in query_result["id"]:
        query_result.remove_rows(query_result.loc_indices[child_id])

query_result = unique(query_result)

## Calculate image parameters

In [97]:
query_result["flux"] = 10 ** (
    -query_result["magnitude"] / 2.5
)  # relative to V-band reference flux
query_result["flux"] = query_result["flux"].round(5)

degrees_per_pixel = (view_radius / (image_pixels / 2)).to(u.deg).value

airy_disk_minimum = 23 * u.arcmin / 2  # based on Vega spread in SIMBAD image
airy_disk_pixels = airy_disk_minimum.to(u.deg) / (degrees_per_pixel * u.deg)

# assume the airy disk is at 3x standard deviation of the Gaussian
std_dev = airy_disk_pixels / 3

# standard deviation is "diameter" whilst airy is "radius"
std_dev *= 2

query_result["scaled_flux"] = np.log10(query_result["flux"])
query_result["scaled_flux"] = (
    query_result["scaled_flux"] - np.min(query_result["scaled_flux"]) + 0.2
)
query_result["scaled_flux"] /= np.max(query_result["scaled_flux"])
query_result["scaled_flux"] = query_result["scaled_flux"].round(5)


def get_intensity(radius, flux, sigma):
    """
    How much light is observed from a star at some radius away from it
    """
    exponential = np.exp(-(radius**2) / (sigma**2))

    return flux * exponential

In [98]:
query_result.sort("magnitude")
query_result

id,ra,dec,magnitude,spectral_type,object_type,name,flux,scaled_flux
,deg,deg,,,,,,
object,float64,float64,float64,str1,object,str71,float64,float64
* bet Ori,78.63447,-8.20164,0.13,B,s*b,Rigel,0.88716,1.0
* alf CMi,114.8255,5.22499,0.37,F,SB*,Procyon/Procyon A,0.71121,0.96742
* alf Ori,88.79294,7.40706,0.42,M,s*r,Betelgeuse,0.6792,0.96063
* gam Ori,81.28276,6.3497,1.64,B,V*,Bellatrix,0.2208,0.79502
* eps Ori,84.05339,-1.20192,1.69,B,s*b,Alnilam,0.21086,0.78823
* zet Ori,85.18969,-1.94257,1.77,O,**,Alnitak,0.19588,0.77737
* kap Ori,86.93912,-9.6696,2.06,B,s*b,Saiph,0.14997,0.738
* del Ori,83.00167,-0.2991,2.41,O,**,Mintaka,0.10864,0.69049


## Add stars to image

In [99]:
image = np.zeros((frames, 3, image_pixels, image_pixels))
backgrounds = []
if frames < 2: dt_astropy = [dt_astropy]
for i in range(frames):
    utc_datetime = dt_astropy[i].to_datetime().replace(tzinfo=ZoneInfo("UTC"))
    local_datetime = utc_datetime.astimezone(tz=timezone)

    day_percentage = timedelta(
        hours=local_datetime.hour, minutes=local_datetime.minute
    ).total_seconds() / (24 * 60 * 60)

    backgroundcolour = np.array(backgroundcolours(day_percentage)[:-1])
    x = np.ones_like(image[i]).T * backgroundcolour
    image[i] = np.swapaxes(np.ones_like(image[i]).T * backgroundcolour, 0, -1)
    backgrounds.append(backgroundcolour)

In [100]:
maxradius = np.ceil(10 * std_dev)  # calculate contribution out to 5 standard deviations
radius_vector = np.arange(-maxradius, maxradius + 1).astype(int)
area = np.array(
    np.meshgrid(radius_vector, radius_vector)
)  # mesh of points which will map to around the star

radial_distance = np.sqrt(
    area[0] ** 2 + area[1] ** 2
)  # radius measurement at each of the meshgrid points

unique_radii = np.unique(radial_distance)

# all of the locations where each unique radius is found
radius_locations = [
    np.array(np.where(radial_distance == radius)).T for radius in unique_radii
]

wcs_by_frame = []
if frames < 2: ra_dec = [ra_dec]
for i in range(frames):
    wcs = WCS(naxis=2)
    wcs.wcs.crpix = [image_pixels / 2] * 2
    wcs.wcs.cdelt = [degrees_per_pixel, degrees_per_pixel]
    wcs.wcs.crval = [ra_dec[i].ra.value, ra_dec[i].dec.value]
    wcs.wcs.ctype = ["RA", "DEC"]
    wcs.wcs.cunit = [u.deg, u.deg]
    wcs_by_frame.append(wcs)

    utc_datetime = dt_astropy[i].to_datetime().replace(tzinfo=ZoneInfo("UTC"))
    local_datetime = utc_datetime.astimezone(tz=timezone)
    day_seconds = int(timedelta(
        hours=local_datetime.hour, minutes=local_datetime.minute
    ).total_seconds())
    visible_stars = query_result[query_result['magnitude'] < magnitude_mapping[day_seconds]]

    print(f"{local_datetime.time()} max mag={magnitude_mapping[day_seconds]:.2f} # stars={len(visible_stars)}")

    for ra, dec, flux, spectral_type in visible_stars[["ra", "dec", "scaled_flux", 'spectral_type']]:
        skycoord = SkyCoord(ra=ra, dec=dec)  # star location
        x, y = np.round(skycoord.to_pixel(wcs)).astype(int)

        # do all points at some given radius at once
        for radius, points in zip(unique_radii, radius_locations):
            brightness = get_intensity(radius, flux, std_dev)

            for area_x, area_y in points:
                # get the pixel locations where this radius applies
                x_ = x + area[1][area_x, area_y]
                y_ = y + area[0][area_x, area_y]

                if (0 <= x_ < image.shape[-1]) and (0 <= y_ < image.shape[-2]):
                    current_rgb = image[i, :, y_, x_]
                    star_rgb = spectral_colours[spectral_type]
                    star_weight = brightness
                    sky_weight = 1 - brightness
                    new_rgb = np.average(
                        [current_rgb, star_rgb],
                        weights=[sky_weight, star_weight],
                        axis=0,
                    )
                    image[i, :, y_, x_] = new_rgb

19:00:00 max mag=3.67 # stars=19
19:05:00 max mag=3.72 # stars=21
19:09:59.999999 max mag=3.77 # stars=23
19:14:59.999999 max mag=3.82 # stars=24
19:19:59.999999 max mag=3.88 # stars=24
19:24:59.999999 max mag=3.93 # stars=27
19:29:59.999998 max mag=3.99 # stars=28
19:34:59.999998 max mag=4.04 # stars=29
19:39:59.999998 max mag=4.10 # stars=31
19:44:59.999997 max mag=4.16 # stars=33
19:49:59.999997 max mag=4.21 # stars=34
19:54:59.999997 max mag=4.27 # stars=34
19:59:59.999997 max mag=4.32 # stars=39
20:04:59.999996 max mag=4.38 # stars=41
20:09:59.999996 max mag=4.43 # stars=44
20:14:59.999996 max mag=4.49 # stars=50
20:19:59.999996 max mag=4.54 # stars=54
20:24:59.999995 max mag=4.60 # stars=57
20:29:59.999995 max mag=4.66 # stars=60
20:34:59.999995 max mag=4.71 # stars=63
20:39:59.999994 max mag=4.77 # stars=67
20:44:59.999994 max mag=4.82 # stars=71
20:49:59.999994 max mag=4.88 # stars=72
20:54:59.999994 max mag=4.93 # stars=75


## Plot result

In [101]:
gif_frame_paths = []
for i, (frametime, wcs) in enumerate(zip(dt_astropy, wcs_by_frame)):
    fig, ax = pu.fig_setup(wcs=wcs, plot_kwargs={"frame_class": EllipticalFrame})

    utc_datetime = frametime.to_datetime().replace(tzinfo=ZoneInfo("UTC"))
    local_datetime = utc_datetime.astimezone(tz=timezone)

    day_percentage = timedelta(
        hours=local_datetime.hour, minutes=local_datetime.minute
    ).total_seconds() / (24 * 60 * 60)

    cmap = LinearSegmentedColormap.from_list(
        "sky", [backgroundcolours(day_percentage), foregroundcolour]
    )

    rgb = np.swapaxes(image[i], 0, -1)
    rgb = np.swapaxes(rgb, 0,1)
    im = ax.imshow(rgb, vmin=0, vmax=1)#, cmap=cmap)

    pu.style_wcs_axes(ax, axis_ticks=(False, False))
    ax.invert_xaxis()

    location_string = f"{location}"
    datetime_string = local_datetime.strftime("%Y-%m-%d %H:%M %Z")
    altaz_string = f"Altitude: {observation_point[0].to_string(format="latex")}, Azimuth: {observation_point[1].to_string(format="latex")}, FOV: {(2*view_radius).to_string(format="latex")}"
    ax.set_title(f"{location_string}\n{datetime_string}\n{altaz_string}")

    fname = f"SkySim_{i}.png"
    pu.save_fig(fig, fname, directory=image_directory)
    gif_frame_paths.append(fname)

/home/taiwithers/projects/skysim/mvp/SkySim_0.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_1.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_2.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_3.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_4.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_5.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_6.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_7.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_8.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_9.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_10.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_11.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_12.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_13.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_14.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_15.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_16.png saved.
/home/taiwithers/project

In [102]:
gif_sequence = Image()
for fname in gif_frame_paths:
    frame = Image(filename=fname)
    frame.delay = int(100 / fps)
    gif_sequence.sequence.append(frame)

gif_sequence.save(filename=gif_fname)

In [103]:
if delete_frames:
    for fname in gif_frame_paths:
        os.remove(fname)